In [2]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd


In [73]:
def read_txt_file(filename):

    inputs = []
    outputs = []
    
    with open(filename, 'r') as file:
        lines = file.readlines()
        current_input = []
        current_output = []
        # Line by line we strip and split all values 
        for line in lines:
            values = line.strip().split()
            # If not the end of a record 
            if values and values[0] != '1.0':
                # We add the whole line (if invalid its going to be set to 0) 
                # First 12 set of values are input, the next 12 are output
                # I only got this by looking through the original matlab file and inferencing this fact
                # I could be wrong so maybe I'll ask the professor 
                input_values = [float(val) if val else 0 for val in values[:12]]
                output_values = [float(val) if val else 0 for val in values[12:]]
                current_input.append(input_values)
                current_output.append(output_values)
            # We're at the end
            elif values and values[0] == '1.0':
                inputs.append(current_input)
                outputs.append(current_output)
                current_input = []
                current_output = []
    return inputs, outputs


# Read the files
train_inputs, train_outputs = read_txt_file('ae.train')
test_inputs, test_outputs = read_txt_file('ae.test')

train_outputs = []
for i in range(269):
    speaker_index = (i // 30) + 1  # Assuming 9 speakers, each with 30 time series
    l = len(train_inputs[i])
    teacher = np.zeros((l, 9))
    teacher[:, speaker_index - 1] = 1  # One-hot encoding for speaker index
    train_outputs.append(teacher)

# Create teacher signals for test data
test_outputs = []
speaker_index = 1
block_counter = 0
block_lengths = [31, 35, 88, 44, 29, 24, 40, 50, 29]  # Assuming the same block lengths as in MATLAB code
for i in range(370):
    block_counter += 1
    if block_counter > block_lengths[speaker_index - 1]:
        speaker_index += 1
        block_counter = 1
    l = len(test_inputs[i])
    teacher = np.zeros((l, 9))
    teacher[:, speaker_index - 1] = 1  # One-hot encoding for speaker index
    test_outputs.append(teacher)

# READ: Different recording have different lengths 
# Do we a) shorten the recordings to the shortest one, 
# b) pad the recordings to the longest one, or
# c) something else?
# For now I'm doing b) but keep that in mind
max_len_train_inputs = max(len(ts) for ts in train_inputs)
max_len_train_outputs = max(len(ts) for ts in train_outputs)
max_len_test_inputs = max(len(ts) for ts in test_inputs)
max_len_test_outputs = max(len(ts) for ts in test_outputs)

train_inputs = [np.pad(ts, ((0, max_len_train_inputs - len(ts)), (0, 0)), mode='constant', constant_values=0) for ts in train_inputs]
train_outputs = [np.pad(ts, ((0, max_len_train_outputs - len(ts)), (0, 0)), mode='constant', constant_values=0) for ts in train_outputs]
test_inputs = [np.pad(ts, ((0, max_len_test_inputs - len(ts)), (0, 0)), mode='constant', constant_values=0) for ts in test_inputs]
test_outputs = [np.pad(ts, ((0, max_len_test_outputs - len(ts)), (0, 0)), mode='constant', constant_values=0) for ts in test_outputs]

train_inputs = np.array(train_inputs)
test_inputs = np.array(test_inputs)
train_outputs = np.array(train_outputs)
test_outputs = np.array(test_outputs)



In [74]:
# BOOM 
print(train_inputs)
print(test_inputs)
print(train_outputs.shape)
print(test_outputs.shape)

[[[ 1.860936 -0.207383  0.261557 ... -0.306756 -0.213076  0.088728]
  [ 1.891651 -0.193249  0.235363 ... -0.289431 -0.247722  0.093011]
  [ 1.939205 -0.239664  0.258561 ... -0.314894 -0.227908  0.074638]
  ...
  [ 1.370862 -0.621346  0.600771 ... -0.105327 -0.193044  0.119152]
  [ 1.307289 -0.600573  0.620979 ... -0.167528 -0.175811  0.088565]
  [ 1.334578 -0.542157  0.558104 ... -0.188285 -0.13861   0.054478]]

 [[ 1.462484  0.174066  0.505133 ... -0.23763   0.120636  0.193254]
  [ 1.309815  0.120183  0.503046 ... -0.231087  0.121053  0.202386]
  [ 1.418207  0.015721  0.589994 ... -0.224317  0.175298  0.15667 ]
  ...
  [ 0.        0.        0.       ...  0.        0.        0.      ]
  [ 0.        0.        0.       ...  0.        0.        0.      ]
  [ 0.        0.        0.       ...  0.        0.        0.      ]]

 [[ 1.160837  0.078806  0.237706 ...  0.028707  0.07482   0.146297]
  [ 1.217979 -0.043693  0.378571 ...  0.03897   0.049702  0.164537]
  [ 1.234654 -0.107083  0.504189

In [49]:
import pandas as pd

train_inputs_2d = train_inputs.reshape(-1, 12)  # Flatten the first two dimensions
train_inputs_df = pd.DataFrame(train_inputs_2d, columns=[f'feature_{i}' for i in range(1, 13)])
train_inputs_df['time_series'] = np.repeat(np.arange(train_inputs.shape[0]), train_inputs.shape[1])
train_inputs_df['time_step'] = np.tile(np.arange(train_inputs.shape[1]), train_inputs.shape[0])
train_inputs_df_X = train_inputs_df.set_index(['time_series', 'time_step'])
train_outputs_2d = train_outputs.reshape(-1, 9)
train_outputs_df = pd.DataFrame(train_outputs_2d, columns=[f'speaker_{i}' for i in range(1, 10)])
train_outputs_df['time_series'] = np.repeat(np.arange(train_outputs.shape[0]), train_outputs.shape[1])
train_outputs_df['time_step'] = np.tile(np.arange(train_outputs.shape[1]), train_outputs.shape[0])
train_outputs_df_Y = train_outputs_df.set_index(['time_series', 'time_step'])

AttributeError: 'tuple' object has no attribute 'reshape'

In [51]:
test_inputs_2d = test_inputs.reshape(-1, 12)  # Flatten the first two dimensions
test_inputs_df = pd.DataFrame(test_inputs_2d, columns=[f'feature_{i}' for i in range(1, 13)])
test_inputs_df['time_series'] = np.repeat(np.arange(test_inputs.shape[0]), test_inputs.shape[1])
test_inputs_df['time_step'] = np.tile(np.arange(test_inputs.shape[1]), test_inputs.shape[0])
test_inputs_df_X = test_inputs_df.set_index(['time_series', 'time_step'])
test_outputs_2d = test_outputs.reshape(-1, 9)
test_outputs_df = pd.DataFrame(test_outputs_2d, columns=[f'speaker_{i}' for i in range(1, 10)])
test_outputs_df['time_series'] = np.repeat(np.arange(test_outputs.shape[0]), test_outputs.shape[1])
test_outputs_df['time_step'] = np.tile(np.arange(test_outputs.shape[1]), test_outputs.shape[0])
test_outputs_df_Y = test_outputs_df.set_index(['time_series', 'time_step'])


In [52]:
train_inputs_df_X = train_inputs_df_X.reset_index()
train_inputs_df_X.head(10)

,index,time_series,time_step,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12
0,0,0,0,1.860936,-0.207383,0.261557,-0.214562,-0.171253,-0.118167,-0.277557,0.025668,0.126701,-0.306756,-0.213076,0.088728
1,1,0,1,1.891651,-0.193249,0.235363,-0.249118,-0.112890,-0.112238,-0.311997,-0.027122,0.171457,-0.289431,-0.247722,0.093011
2,2,0,2,1.939205,-0.239664,0.258561,-0.291458,-0.041053,-0.102034,-0.383300,0.019013,0.169510,-0.314894,-0.227908,0.074638
3,3,0,3,1.717517,-0.218572,0.217119,-0.228186,-0.018608,-0.137624,-0.403318,-0.009643,0.164607,-0.323267,-0.210105,0.098098
4,4,0,4,1.741191,-0.279891,0.196583,-0.236377,-0.032012,-0.090612,-0.363134,-0.012571,0.124298,-0.351171,-0.216545,0.113899
5,5,0,5,1.684695,-0.311977,0.195453,-0.231970,-0.068670,-0.003822,-0.341940,-0.008826,0.085097,-0.364329,-0.204794,0.101838
6,6,0,6,1.637373,-0.336227,0.152766,-0.223842,-0.026278,-0.009157,-0.363866,-0.003117,0.055479,-0.358107,-0.181643,0.082056
7,7,0,7,1.643283,-0.349773,0.131553,-0.154519,-0.035292,0.023719,-0.381399,-0.021189,0.020397,-0.340491,-0.156417,0.080884
8,8,0,8,1.607030,-0.382745,0.179038,-0.115949,-0.060406,0.057800,-0.364642,-0.069230,-0.019788,-0.355996,-0.115129,0.131928
9,9,0,9,1.617907,-0.527367,0.179878,-0.083292,0.031747,0.081424,-0.418227,-0.081175,-0.022385,-0.337660,-0.103184,0.102266


In [54]:
train_outputs_df_Y = train_outputs_df_Y.reset_index()
train_outputs_df_Y.head(5)

,time_series,time_step,speaker_1,speaker_2,speaker_3,speaker_4,speaker_5,speaker_6,speaker_7,speaker_8,speaker_9
0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
mm = MinMaxScaler()
ss = StandardScaler()

# Normalize all values
X_ss = ss.fit_transform(train_inputs_df_X)
y_mm = mm.fit_transform(train_outputs_df_Y) 

ModuleNotFoundError: No module named 'sklearn'

In [158]:
train_inputs_df_X.to_csv('train_inputs.csv')
train_outputs_df_Y.to_csv('train_outputs.csv')

# CONVER THE TIME SERIES DATA IN A PROPER FORMAT FOR A TENSOR STACK
time_series_data = train_inputs_df_X#.drop(['time_series', 'time_step'], axis=1)#train_inputs_df_X[['time_series', 'time_step']]
time_series_tensors = []
for ts_id, group in time_series_data.groupby('time_series'):
    group = group.drop('time_series', axis=1)
    group = group.values.astype(np.float32)
    time_series_tensors.append(torch.from_numpy(group))

# Essentially we just leave
feature_data = train_inputs_df_X.drop(['time_series', 'time_step'], axis=1)


In [159]:
print(feature_data)

       index  feature_1  feature_2  feature_3  feature_4  feature_5  \
0          0   1.860936  -0.207383   0.261557  -0.214562  -0.171253   
1          1   1.891651  -0.193249   0.235363  -0.249118  -0.112890   
2          2   1.939205  -0.239664   0.258561  -0.291458  -0.041053   
3          3   1.717517  -0.218572   0.217119  -0.228186  -0.018608   
4          4   1.741191  -0.279891   0.196583  -0.236377  -0.032012   
...      ...        ...        ...        ...        ...        ...   
12638  12638   0.000000   0.000000   0.000000   0.000000   0.000000   
12639  12639   0.000000   0.000000   0.000000   0.000000   0.000000   
12640  12640   0.000000   0.000000   0.000000   0.000000   0.000000   
12641  12641   0.000000   0.000000   0.000000   0.000000   0.000000   
12642  12642   0.000000   0.000000   0.000000   0.000000   0.000000   

       feature_6  feature_7  feature_8  feature_9  feature_10  feature_11  \
0      -0.118167  -0.277557   0.025668   0.126701   -0.306756   -0.213

In [160]:
train_outputs_df_Y.head(5)

,time_series,time_step,speaker_1,speaker_2,speaker_3,speaker_4,speaker_5,speaker_6,speaker_7,speaker_8,speaker_9
0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [161]:
target_features = []
for ts_id, group in train_outputs_df_Y.groupby('time_series'):
    group = group.drop('time_series', axis=1)
    group = group.values.astype(np.float32)
    target_features.append(torch.from_numpy(group))

features = torch.stack(target_features)

In [162]:
print(target_features)

[tensor([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 2.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 3.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 4.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 5.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 6.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 7.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 8.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 9.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [10.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [11.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [12.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [13.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [14.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [15.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [16.,  1.,  0.,  0.,  0.,  0., 

In [163]:
# Transpose things to just grab the speaker ID
melted_df = train_outputs_df_Y.melt(id_vars=['time_series', 'time_step'], 
                                    value_vars=[f'speaker_{i}' for i in range(1, 10)], 
                                    var_name='speaker', value_name='is_speaking')

speaker_df = melted_df[melted_df['is_speaking'] == 1.0]
speaker_df['speaker'] = speaker_df['speaker'].str.extract('(\d)').astype(int)

speaker_df = speaker_df.drop(columns='is_speaking')
speaker_df = speaker_df.drop(columns='time_step')
speaker_df = speaker_df.drop_duplicates()
speaker_df = speaker_df.reset_index(drop=True)

<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_35596/2016340290.py:7: SyntaxWarning: invalid escape sequence '\d'
  speaker_df['speaker'] = speaker_df['speaker'].str.extract('(\d)').astype(int)
/tmp/ipykernel_35596/2016340290.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  speaker_df['speaker'] = speaker_df['speaker'].str.extract('(\d)').astype(int)


In [164]:
print(speaker_df)

     time_series  speaker
0              0        1
1              1        1
2              2        1
3              3        1
4              4        1
..           ...      ...
264          264        9
265          265        9
266          266        9
267          267        9
268          268        9

[269 rows x 2 columns]


In [165]:
# Convert data to PyTorch tensors
from torch.autograd import Variable 
X = train_inputs_df_X.iloc[:, :].values
Y = train_outputs_df_Y.iloc[:, :].values

input_tensor = torch.stack(time_series_tensors)
print(input_tensor.shape)

targets = torch.from_numpy(speaker_df['speaker'].values).long()
print(targets.shape)

torch.Size([269, 47, 14])
torch.Size([269])


In [166]:
# Version 2
# Hyperparameters

class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, output_size):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes  # should be 9
        self.num_layers = num_layers  # up to you
        self.input_size = input_size  # should 12
        self.hidden_size = hidden_size  # hidden neurons

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)  # lstm
        self.fc_1 = nn.Linear(hidden_size, 128)  # fully connected 1
        self.fc = nn.Linear(128, num_classes)  # fully connected last layer

        self.relu = nn.ReLU()

    def forward(self, x):
        # Get the batch size from the input tensor
        batch_size = x.size(0)  
        h_0 = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        c_0 = torch.zeros(self.num_layers, batch_size, self.hidden_size)

        # Propagatiooon time
        output, (hn, cn) = self.lstm(x, (h_0, c_0))  
        output = output.contiguous().view(batch_size, -1, self.hidden_size)

        out = self.relu(output[:, -1, :])  # Apply ReLU activation to the last output
        out = self.fc_1(out)  # First Dense layer
        out = self.relu(out)  # ReLU activation
        out = self.fc(out)  # Final Output layer

        return out

In [170]:
# Hyperparameters
num_classes = 1  # How many features we detecting??? (1)
input_size = 14  # Should be 12
hidden_size = 64  # Size of the hidden state in the LSTM
num_layers = 2  # Number of LSTM layers
output_size = 1  # Size of the output (in your case, it's 1 since you have one output feature)
num_epochs = 100 

# Initialize the model
model = LSTM1(num_classes, input_size, hidden_size, num_layers, output_size)
# Loss function (assuming it's a regression problem)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [171]:
input_data = input_tensor.float()
targets = targets.float()

In [172]:
# THIS WORKS
for epoch in range(num_epochs):
    # Perform one training step
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(input_data)
    loss = criterion(outputs, targets)
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 

/home/kikis/.local/lib/python3.12/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([269])) that is different to the input size (torch.Size([269, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 0, loss: 30.89599
Epoch: 10, loss: 26.60159


KeyboardInterrupt: 